In [1]:
ds_path = '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data'
ds_path

'/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data'

In [2]:
import os
import pandas as pd

In [3]:
classes = list(filter(lambda x: '.' not in x, [f for f in os.listdir(ds_path) if os.path.isdir(ds_path)]))
classes

['253_Cursive_Chuyig',
 '221_Pedri_Tsegdrig',
 '224_Pedri_Petsug',
 '114_Uchan_Shachen',
 '122_Woodblock_Sugthung',
 '113_Uchan_Sugring',
 '214_Druma_Sugring',
 '222_Pedri_Godrig',
 '212_Druma_Sugthung',
 '242_Tsugma_Sugdring',
 '232_Drari_Sugthung',
 '252_Cursive_Tsumachug',
 '117_Uchan_Handwritten',
 '202_EarlyDruma_Regular',
 '104_EarlyUchan_Thin',
 '103_EarlyUchan_Bold',
 '111_Uchan_Regular',
 '231_Drari_Regular',
 '223_Pedri_Peri',
 '244-3_Tsugma_Tsugchung',
 '233_Drari_Sugdring',
 '213_Druma_Sugdring',
 '123_Woodblock_Sugring',
 '121_Woodblock_Regular',
 '112_Uchan_Sugthung',
 '211_Druma_Dhumri',
 '215_Druma_Drutsa',
 '244-2_Tsugma_Tsugthung',
 '134_Modern_Sarchung',
 '132_Modern_Text']

In [8]:
all_samples = {}

for class_ in classes:
    path = f"{ds_path}/{class_}"
    samples_pathes = list(filter(lambda x: '.jpg' in x.lower(), [f"{ds_path}/{class_}/{f}" for f in os.listdir(path) if os.path.isdir(path)]))
    all_samples[class_] = samples_pathes
            
all_samples

{'253_Cursive_Chuyig': ['/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/W4PD1207_I4PD12380288_4.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I8LS193070071_6.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I4PD12380330_5.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I4PD12400121_3.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I4PD12380288_0.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I4PD12460453_2.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I8LS197350086_3.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/I8LS196990008_2.jpg',
  '/Users/hjklvfr/Projects/ocr/data/ScriptClassification/Data/253_Cursive_Chuyig/W4PD1207_I4PD12380348_1.jpg',
  '/Users/hjklvfr/Projects/ocr/data/Scr

In [4]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

In [14]:
class BDRCScriptClassification(Dataset):
    def __init__(self, path, transform=None):
        self.transform = transform
        self.paths = []
        self.labels = []

        classes = list(filter(lambda x: '.' not in x, [f for f in os.listdir(ds_path) if os.path.isdir(ds_path)]))
        
        self.labels_to_idx = {
            label: i for i, label in enumerate(sorted(set(classes)))
        }

        for class_ in classes:
            path = f"{ds_path}/{class_}"
            samples_pathes = list(filter(lambda x: '.jpg' in x.lower(), [f"{ds_path}/{class_}/{f}" for f in os.listdir(path) if os.path.isdir(path)]))
            self.paths += samples_pathes
            self.labels += [class_] * len(samples_pathes)
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        image = Image.open(path).convert('1')
        if self.transform:
            image = self.transform(image)
            # print(f"image shape: {image.shape}")
        label = self.labels[idx]
        return image, torch.tensor(self.labels_to_idx[label], dtype=torch.long)


transform = transforms.ToTensor()
dss = BDRCScriptClassification(ds_path, transform)
dss.__len__()
        
        

13230

In [6]:
from typing import Sequence, Tuple
from sklearn.model_selection import train_test_split
import numpy as np
import torch

def train_val_test_split(dataset: Dataset) -> Tuple[torch.utils.data.Subset, torch.utils.data.Subset, torch.utils.data.Subset]:
    train_idx, temp_idx = train_test_split(
        np.arange(len(dss)),
        test_size=0.2,
        stratify=dss.labels,
        random_state=42
    )

    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,
        stratify=np.array(dss.labels)[temp_idx],
        random_state=42
    )

    train_ds = torch.utils.data.Subset(dss, train_idx)
    val_ds = torch.utils.data.Subset(dss, val_idx)
    test_ds = torch.utils.data.Subset(dss, test_idx)

    return (train_ds, val_ds, test_ds)

In [7]:
# import torch

# # dss = BDRCScriptClassification(ds_path, transform)

# images = torch.stack([x[0] for x in train_ds], dim=0)
# mean = torch.mean(images)
# std = torch.std(images)

# (mean, std)

mean = 0.4005

std = 0.7993

In [8]:
# from collections import Counter

# def count_class_balances(subset):
#     lables = np.array(subset.dataset.labels)[subset.indices]
#     return Counter(lables)

# count_class_balances(test_ds)

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

torch.backends.mps.is_available()

True

In [15]:
batch_size = 1
lr = 1e-3
epochs = 5

device = torch.device('mps')

transform = transforms.Compose([
    transforms.Resize(size=(600,600)),
    transforms.ToTensor(),
    transforms.Normalize((.4005), (.7993))
])

dataset_overall = BDRCScriptClassification(ds_path, transform)
train_ds, val_ds, test_ds = train_val_test_split(dataset_overall)

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size, shuffle=False)

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(
            in_features=64*150*150,
            out_features=512
        )
        self.fc2 = nn.Linear(in_features=512, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=30)
    
    def forward(self, x):
        # print(x.shape)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        # print(x.shape)

        x = torch.flatten(x, start_dim=1)
        # print(x.shape)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x
    
model = CNNModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()

    running_loss = 0.
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(output, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{epochs} "
          f"train loss: {epoch_loss:.4f} "
          f"train acc: {epoch_acc:.4f}")

model.eval()
correct = 0
total = 0

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    output = model(images)
    _, preds = torch.max(output, dim=1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

test_acc = correct / total
print(f"Test accuracy: {test_acc:.4f}")


KeyboardInterrupt: 